<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [595]:
import pandas as pd
import psycopg2

In [596]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 


In [597]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [598]:
# текст запроса
# создаем лямбда-функцию, содержащую стандартный запрос на подсчет числа строк в таблице, параметр - название таблицы
counter = lambda x: f'''
    SELECT 
        COUNT(*)
    FROM
        public.{x}
'''
#применяем функцию к таблице vacancies
query_3_1 = counter('vacancies')

In [599]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\1114379196.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [600]:
# текст запроса

query_3_2 = counter('employers')


In [601]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\3062646138.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_2, connection)


,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [602]:
# текст запроса
query_3_3 = counter('areas')

In [603]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\928149064.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_3, connection)


,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [604]:
# текст запроса
query_3_4 = counter('industries')

In [605]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\2101691108.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_4, connection)


,count
0,294


***

### выводы по предварительному анализу данных
База довольно репрезентативна -- большое колчиество вакансий, охвачено 1362 локации, представлено 294 сферы деятельности, большое количество работодателей. В среднем в выборке 160 вакансий на отрасль, порядка 18 компаний на регион.


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [606]:
# текст запроса
query_4_1 = f'''
    SELECT 
        a.name as area,
        COUNT(v.id) as cnt
    FROM 
        public.areas as a,
        public.vacancies as v
    WHERE
        a.id = v.area_id  -- соединяем таблицы по id региона
    GROUP BY area
    ORDER BY cnt DESC
'''

In [607]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\2457173557.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_1, connection)


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [608]:
# текст запроса
query_4_2 = f'''
    SELECT 
        count(*)
    FROM 
        public.vacancies
    WHERE (salary_from IS NOT NULL) OR (salary_to IS NOT NULL)
'''

In [609]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\970655580.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_2, connection)


,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [610]:
# текст запроса
query_4_3 = f'''
    SELECT
        ROUND(AVG(salary_from))::int as salary_from_avg, -- приводим к целому после округления
        ROUND(AVG(salary_to))::int as salary_to_avg
    FROM 
        public.vacancies
'''

In [611]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\2500607024.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_3, connection)


,salary_from_avg,salary_to_avg
0,71065,110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [612]:
# текст запроса
query_4_4 = f'''
    SELECT
        schedule,
        employment,
        COUNT(*) as cnt
    FROM 
        public.vacancies
    GROUP BY schedule, employment
    ORDER BY cnt DESC
'''

In [613]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\2022935265.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_4, connection)


,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [614]:
# текст запроса
query_4_5 = f'''
    SELECT
        experience
    FROM 
        public.vacancies
    GROUP BY experience
    ORDER BY COUNT(*)
'''

In [615]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\2316260449.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_5, connection)


,experience
0,Более 6 лет
1,Нет опыта
2,От 3 до 6 лет
3,От 1 года до 3 лет


***

### выводы по детальному анализу вакансий

В дополнительном анализе (см. в конце ноутбука) посчитано в %% какая доля вакансий приходится на крупнейшие по количеству вакансий 20 регионов по отдельности и совокупно. С одной стороны ТОП20 регионов совокупно составляют 60% всех вакансий, что говорит о достаточно высококой концентрации на небольшом количестве крупных локаций. С другой стороны, у самогой крупной локации - Москвы - всего 10% всех вакансий; а у 32 локаций доля от 0.5% до 3% от общего числа. Это говорит об уммеренной распределнности вакансий по географическому принципу и об отсутствии подавляющего доминирования отдельных регионов.

Аналогичный анализ по распределению вакаансий по работодателям показывает, что у выборки хорошее распрееделение по работодателям. У крупнейшего работодателя - Янедкса меньше 4% от общего числа. А ТОП50 работодателей составляет всего 18% от общего числа вакансий.

Из всего этого можно сделать вывод, что выборка достаточно репрезентативна. 

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [616]:
# текст запроса
query_5_1 = f'''
    -- в первом блоке - работодатель на 1 месте
    (
    SELECT
        e.name as employer
    FROM 
        public.vacancies as v,
        public.employers as e
    WHERE v.employer_id = e.id
    GROUP BY employer -- группировка для сортировки по числу вакансий
    ORDER BY COUNT(*) DESC  -- сама сортировка
    LIMIT 1 -- оставляем того кто на первом месте
    )
    
    UNION ALL 
    
    -- во втором блоке - работодатель на 5 месте, структура блока аналогична первому
    (
    SELECT
        e.name as employer
    FROM 
        public.vacancies as v,
        public.employers as e
    WHERE v.employer_id = e.id
    GROUP BY employer
    ORDER BY COUNT(*) DESC
    OFFSET 4 LIMIT 1 -- пропускаем первые 4 строки, оставляем только 5ую
    )
'''

In [617]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\3834399874.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_1, connection)


,employer
0,Яндекс
1,Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [618]:
# текст запроса

query_5_2 = f'''
    SELECT
        a.name as area,
        COUNT(DISTINCT e.id) as cnt_eployers, -- добавляем DISTINCT чтобы убрать повторные вхождения
        COUNT(DISTINCT v.id) as cnt_vacancies -- аналогично
    FROM 
        public.areas as a
    LEFT JOIN         
        public.employers as e 
        ON a.id = e.area
    LEFT JOIN 
        public.vacancies as v 
        ON a.id = v.area_id
    GROUP BY a.id
    ORDER BY 3 ASC, 2 DESC -- искомый регион будет в первой строке
'''

In [619]:
# результат запроса
df = pd.read_sql_query(query_5_2, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\2250345976.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_2, connection)


,area,cnt_eployers,cnt_vacancies
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
...,...,...,...
1357,Алматы,721,1892
1358,Новосибирск,573,2006
1359,Минск,1115,2112
1360,Санкт-Петербург,2217,2851


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [620]:
# текст запроса
query_5_3 = f'''
    SELECT
        e.name as employer,
        COUNT(DISTINCT v.area_id) as cnt_areas -- добавляем DISTINCT чтобы убрать повторные вхождения
    FROM 
        public.employers as e
    LEFT JOIN 
        public.vacancies as v 
        ON v.employer_id = e.id
    GROUP BY e.id
    ORDER BY 2 DESC
'''

In [621]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\201667002.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_3, connection)


,employer,cnt_areas
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23496,Но Смокинг,0
23497,МАСТЕРСКАЯ CREED,0
23498,Dzing Finance,0
23499,КОРС,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [622]:
# текст запроса
query_5_4 = f'''
    SELECT
        COUNT(e.id) as employer_unknown_ind_count
    FROM 
        public.employers as e
    LEFT JOIN -- соединяем слева к таблице работодателей, чтобы найти "пустых" работодателей, кому не сопоставится отрасль
        public.employers_industries as ei 
        ON ei.employer_id = e.id
    WHERE ei.industry_id IS NULL -- условие на "пустоту" отрасли
'''

In [623]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\3493105328.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_4, connection)


,employer_unknown_ind_count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [624]:
# текст запроса
query_5_5 = f'''
    SELECT
        e.name as name
    FROM 
        public.employers as e
    LEFT JOIN 
        public.employers_industries as ei 
        ON ei.employer_id = e.id
    GROUP BY e.name
    HAVING COUNT(DISTINCT ei.industry_id) = 4 -- условие что ровно 4 разных отрасли
    ORDER BY 1
    OFFSET 2 LIMIT 1 -- оставляем только 3 строку
'''

In [625]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\388711819.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_5, connection)


,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [626]:
# текст запроса
query_5_6 = f'''
    SELECT
        COUNT(DISTINCT e.id) -- добавляем DISTINCT чтобы убрать повторные вхождения
    FROM 
        public.employers as e
    LEFT JOIN -- подсоединяем чтобы сопоставить работодателю его отрасль
        public.employers_industries as ei 
        ON ei.employer_id = e.id
    LEFT JOIN -- подсоединяем чтобы поставить фильтр на название отрасли
        public.industries as i 
        ON ei.industry_id = i.id
    WHERE i.name = 'Разработка программного обеспечения'
'''

In [627]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\821628714.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_6, connection)


,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [628]:
# код для получения списка городов-милионников
#million_cities = ['Новосибирск', 'Екатеринбург', 'Нижний Новгород', 'Казань', 'Челябинск', 'Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 
#                  'Воронеж', 'Волгоград', 'Краснодар', 'Москва', 'Санкт-Петербург']

import requests # Импортируем библиотеку requests
from bs4 import BeautifulSoup
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8' # Определяем значение URL страницы для запроса
response = requests.get(url) # Делаем GET-запрос к ресурсу и результат ответа сохраняем в переменной response
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер

# парсим таблицу с указанным классом, забираем внутри нее коллекцию тегов th - это заголовки 
table1 = page.find('table', class_='standard') 
headers = []
for i in table1.find_all('th'):
    headers.append(i.text)

# созадем DataFrame таблицу, для данных о миллиониках, заголовки - считанные парсером
df_mill = pd.DataFrame(columns=headers)

# считываем строки tr, в них ячейки td
for j in table1.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = []
    for  k in row_data:
        # если внутри ячейки сидит ссылка - залезаем в ее текст
        if k.a is not None:
            row.append(k.a.text)
        else:
            row.append(k.text)
    # добавляем считанную строку в DataFrame
    length = len(df_mill)
    df_mill.loc[length] = row
# конвертируем столбец с городами в строку, обрамленную в кругляе скобки, с разделителями запятыми, каждый город в кавычках
cities_list = str(tuple(df_mill['Город']))
print(cities_list)


('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')


In [629]:
# текст запроса
query_5_7 = f'''
    (SELECT
        a.name as region_name,
        COUNT(DISTINCT v.id)
    FROM 
        public.vacancies as v
    LEFT JOIN 
        public.employers as e 
        ON v.employer_id = e.id
    LEFT JOIN
        public.areas as a 
        ON v.area_id = a.id
    WHERE 
        e.name = 'Яндекс' 
        AND 
        a.name IN {cities_list} -- предаем список городов-миллиоников
    GROUP BY a.name
    ORDER BY 2
    )
    
    UNION ALL
    
    (SELECT
        'Total' as region_name, -- добавляем строгу Total
        COUNT(DISTINCT v.id)
    FROM 
        public.vacancies as v
    LEFT JOIN 
        public.employers as e 
        ON v.employer_id = e.id
    LEFT JOIN
        public.areas as a 
        ON v.area_id = a.id
    WHERE 
        e.name = 'Яндекс' 
        AND 
        a.name IN {cities_list}
    )
    
    ORDER BY 2
'''

In [630]:
# результат запроса
df = pd.read_sql_query(query_5_7, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\1965596796.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_7, connection)


,region_name,count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Ростов-на-Дону,25
5,Пермь,25
6,Казань,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

### выводы по анализу работодателей

Требует дальнейшего анализа качество данных, вызываетвопросы к данным то, что регион Москва указан у 5864 работодателей, в то время как у этих же работодателей всего 5 333 вакансии. Возможно, требуется очистка выборки от компаний, не имеющих в текущий момент времени вакансий. 

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [632]:
# текст запроса
query_6_1 = f'''
    SELECT
        COUNT(DISTINCT v.id)
    FROM 
        public.vacancies as v
    WHERE LOWER(v.name) LIKE '%data%' OR LOWER(v.name) LIKE '%данн%'
'''

In [633]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\366702258.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_1, connection)


,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [634]:
# текст запроса
query_6_2 = f'''
    SELECT
        COUNT(DISTINCT v.id)
    FROM 
        public.vacancies as v
    WHERE 
        (
            LOWER(v.name) LIKE '%data scientist%' 
            OR LOWER(v.name) LIKE '%data science%'
            OR LOWER(v.name) LIKE '%исследователь данных%'
            OR (LOWER(v.name) LIKE '%ml%' AND LOWER(v.name) NOT LIKE '%html%') -- убираем те, которые содержат html
            OR LOWER(v.name) LIKE '%machine learning%'
            OR LOWER(v.name) LIKE '%машинн%обучен%'
        )
        AND
        (
            LOWER(v.name) LIKE '%junior%' 
            OR v.experience = 'Нет опыта'
            OR v.employment = 'Стажировка'
        )
'''

In [635]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\2267038862.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_2, connection)


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [636]:
# текст запроса
query_6_3_ = f'''
    SELECT
        COUNT(v.id)
    FROM 
        public.vacancies as v
    WHERE 
        (
            LOWER(v.name) LIKE '%data scientist%' 
            OR LOWER(v.name) LIKE '%data science%'
            OR LOWER(v.name) LIKE '%исследователь данных%'
            OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
            OR LOWER(v.name) LIKE '%machine learning%'
            OR LOWER(v.name) LIKE '%машинн%обучен%'
        )
        AND
        (
            LOWER(v.key_skills) LIKE '%sql%' 
            OR LOWER(v.key_skills) LIKE '%postgres%' 
        )
'''

In [637]:
# результат запроса
df2 = pd.read_sql_query(query_6_3_, connection)
df2

C:\Users\User\AppData\Local\Temp\ipykernel_212\3769163947.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql_query(query_6_3_, connection)


,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [638]:
# текст запроса
query_6_4 = f'''
    SELECT
        COUNT(v.id)
    FROM 
        public.vacancies as v
    WHERE 
        (
            LOWER(v.name) LIKE '%data scientist%' 
            OR LOWER(v.name) LIKE '%data science%'
            OR LOWER(v.name) LIKE '%исследователь данных%'
            OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
            OR LOWER(v.name) LIKE '%machine learning%'
            OR LOWER(v.name) LIKE '%машинн%обучен%'
        )        
        AND
            LOWER(v.key_skills) LIKE '%python%' 
'''

In [639]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\4264665179.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_4, connection)


,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [640]:
# текст запроса
query_6_5 = f'''
    SELECT
        -- число навыков на 1 больше, чем число табуляций
        -- если табуляций в строке n штук, то сработает маска, в которой n табуляций и все до нее, все после нее не сработают
        -- каждое логическое значение конвертируем в int (1 или 0): n истинных дадут n единиц, остальные будут нулями, в сумме получим ровно n
        -- какой длины нужна самая большая маска определяем эмпирически - когда максимум по выборке будет меньше ее длины. Максимум здесь - 19
        AVG  
        (
        (v.key_skills IS NOT NULL)::int +
        (v.key_skills LIKE '%\t%')::int +
        (v.key_skills LIKE '%\t%\t%')::int +
        (v.key_skills LIKE '%\t%\t%\t%')::int +
        (v.key_skills LIKE '%\t%\t%\t%\t%')::int +
        (v.key_skills LIKE '%\t%\t%\t%\t%\t%')::int +
        (v.key_skills LIKE '%\t%\t%\t%\t%\t%\t%')::int +
        (v.key_skills LIKE '%\t%\t%\t%\t%\t%\t%\t%')::int +
        (v.key_skills LIKE '%\t%\t%\t%\t%\t%\t%\t%\t%')::int +
        (v.key_skills LIKE '%\t%\t%\t%\t%\t%\t%\t%\t%\t%')::int +
        (v.key_skills LIKE '%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%')::int +
        (v.key_skills LIKE '%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%')::int +
        (v.key_skills LIKE '%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%')::int +
        (v.key_skills LIKE '%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%')::int +
        (v.key_skills LIKE '%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%')::int +
        (v.key_skills LIKE '%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%')::int +
        (v.key_skills LIKE '%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%')::int +
        (v.key_skills LIKE '%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%')::int +
        (v.key_skills LIKE '%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%')::int +
        (v.key_skills LIKE '%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%')::int +
        (v.key_skills LIKE '%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%\t%')::int
        )
    FROM 
        public.vacancies as v
    WHERE 
        (
            LOWER(v.name) LIKE '%data scientist%' 
            OR LOWER(v.name) LIKE '%data science%'
            OR LOWER(v.name) LIKE '%исследователь данных%'
            OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
            OR LOWER(v.name) LIKE '%machine learning%'
            OR LOWER(v.name) LIKE '%машинн%обучен%'
        )        
'''

In [641]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\3654210517.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_5, connection)


,avg
0,6.406032


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [642]:
# текст запроса
query_6_6 = f'''
    SELECT
        v.experience,
        AVG
        (
        COALESCE(v.salary_from / 2 , 0) +
        COALESCE(v.salary_to / 2, 0) +  
        COALESCE(v.salary_from / 2 , 0) * (v.salary_to IS NULL)::int + -- если salary_to = 0, то тогда нужно взять salary_from целиком, для этого в выражения выше не хватает salary_from / 2
        COALESCE(v.salary_to / 2, 0) * (v.salary_from IS NULL)::int -- аналогично, зеркально
        ) as avg_salary
    FROM 
        public.vacancies as v
    WHERE 
        (
            LOWER(v.name) LIKE '%data scientist%' 
            OR LOWER(v.name) LIKE '%data science%'
            OR LOWER(v.name) LIKE '%исследователь данных%'
            OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
            OR LOWER(v.name) LIKE '%machine learning%'
            OR LOWER(v.name) LIKE '%машинн%обучен%'
        )
        AND
        (
            v.salary_to IS NOT NULL
            OR v.salary_from IS NOT NULL
        )
    GROUP BY v.experience
'''

In [643]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\519194770.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_6, connection)


,experience,avg_salary
0,Нет опыта,74642.857143
1,От 1 года до 3 лет,139674.750000
2,От 3 до 6 лет,243114.666667


***

### выводы по предметному анализу

Лишь 480 вакансий вписываютсчя в выбранные ключевые маски DS из 1771 вакансий связанных с данными. В т.ч. как показывает дополнительный анализ, теряются направления Data analyst, Data engineer. 

Лишь 10% вакансий датасаенс рассчитана на уронеь Junior, в подавляющем числе случаев требуются опытные специалисты.

Доля вакансий DS в которых указан SQL  высока - порядка 42%, что говорит о том, что владение SQL если и не является рыночным стандартом, то как минимум является важным существенным навыкомв арсенале DS специалиста и точно будет существенным плюсом в резюме. 

У 73% вакансий DS ключевым навыком указан python, он очевидно является обязательным рыночным стандартом для Data Science. Учитывая его роль в DS вызывает удивление, что это не величина близкая к 100%. Дополнительный анализ показывает, что во-первых это связано с наличие ошибочно включенных в DS-выборку специалистов по QML и TinyML, без них остается 456 вакансий. У оставшихся зачастую указаны либо более узко-специальные навыки, подразумевающие владение python (computer vision, ML, нейронные сети), либо более широкие, но с аналогичным подтекстом (data science, big data), либо требуются управленческие навыки для менеджерских позиций.

В DS требуется довольно обширный переченьнавыков, в среднем по выборке  указано порядка 6 навыков.

Явно прослеживается значительный рост заработной платы в зависимости от опыта работы, при оценке заработной платы опыт является существенным фактором.

# Общий вывод по проекту

### доп исследования

In [671]:
# текст запроса
query_4_1_plus = f'''
    (SELECT 
        a.name as area,
        COUNT(v.id)::numeric / t.cnt::numeric * 100 as cnt_pct -- считаем долю вакансий в регионе от общего числа в %%
    FROM 
        public.areas as a
    INNER JOIN
        public.vacancies as v
        ON a.id = v.area_id  -- соединяем таблицы по id региона
    CROSS JOIN  -- прописываем TOTAL (число всех вакансий) в каждую строку чтобы суметь посчитать %% в каждой строке
        (   
            SELECT 
                COUNT(v.id) as cnt -- считаем TOTAL число всех вакансий
            FROM 
                public.areas as a,
                public.vacancies as v
            WHERE
                a.id = v.area_id
        ) as t       
    GROUP BY area, t.cnt
    ORDER BY cnt_pct DESC
    LIMIT 20  -- оставляемТОП20 регионов
    )
    
    UNION ALL
    
    (SELECT 
        'TOTAL' as area, -- добавляем строку TOTAL, в которой показываем сколько %% совокупно дают ТОП20 регионов 
        SUM(cnt_pct)
    FROM    -- ниже SELECT - точная копия первой части UNION
            (SELECT 
                a.name as area,
                COUNT(v.id)::numeric / t.cnt::numeric * 100 as cnt_pct
            FROM 
                public.areas as a
            INNER JOIN
                public.vacancies as v
                ON a.id = v.area_id  
            CROSS JOIN
                (   
                    SELECT 
                        COUNT(v.id) as cnt
                    FROM 
                        public.areas as a,
                        public.vacancies as v
                    WHERE
                        a.id = v.area_id
                ) as t       
            GROUP BY area, t.cnt
            ORDER BY cnt_pct DESC
            LIMIT 20
            ) as s
    )
'''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_4_1_plus, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\3024244090.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_1_plus, connection)


,area,cnt_pct
0,Москва,10.840092
1,Санкт-Петербург,5.795069
2,Минск,4.292945
3,Новосибирск,4.077484
4,Алматы,3.845763
5,Екатеринбург,3.451430
6,Нижний Новгород,3.394516
7,Казань,2.876192
8,Краснодар,2.644470
9,Самара,2.325345


In [672]:
# результат запроса
df = pd.read_sql_query(query_4_1_plus, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\3024244090.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_1_plus, connection)


,area,cnt_pct
0,Москва,10.840092
1,Санкт-Петербург,5.795069
2,Минск,4.292945
3,Новосибирск,4.077484
4,Алматы,3.845763
5,Екатеринбург,3.451430
6,Нижний Новгород,3.394516
7,Казань,2.876192
8,Краснодар,2.644470
9,Самара,2.325345


In [677]:
# текст запроса
query_4_1_plus2 = f'''
    SELECT 
        a.name as area,
        COUNT(v.id)::numeric / t.cnt::numeric * 100 as cnt_pct
    FROM 
        public.areas as a
    INNER JOIN
        public.vacancies as v
        ON a.id = v.area_id  -- соединяем таблицы по id региона
    CROSS JOIN
        (   
            SELECT 
                COUNT(v.id) as cnt
            FROM 
                public.areas as a,
                public.vacancies as v
            WHERE
                a.id = v.area_id
        ) as t       
    GROUP BY area, t.cnt
    HAVING COUNT(v.id)::numeric / t.cnt::numeric * 100 > 0.5 -- выведем те локации, в которых доля больше 0,5% от общего числа
    ORDER BY cnt_pct DESC
'''

In [678]:
# результат запроса
df = pd.read_sql_query(query_4_1_plus2, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\3061924742.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_1_plus2, connection)


,area,cnt_pct
0,Москва,10.840092
1,Санкт-Петербург,5.795069
2,Минск,4.292945
3,Новосибирск,4.077484
4,Алматы,3.845763
5,Екатеринбург,3.451430
6,Нижний Новгород,3.394516
7,Казань,2.876192
8,Краснодар,2.644470
9,Самара,2.325345


In [685]:
# текст запроса
query_4_2_plus = f'''
    (SELECT 
        e.name as employer,
        COUNT(v.id)::numeric / t.cnt::numeric * 100 as cnt_pct -- считаем долю вакансий в регионе от общего числа в %%
    FROM 
        public.employers as e
    INNER JOIN
        public.vacancies as v
        ON e.id = v.employer_id  -- соединяем таблицы по id региона
    CROSS JOIN  -- прописываем TOTAL в каждую строку чтобы суметь посчитать %% в каждой строке
        (   
            SELECT 
                COUNT(v.id) as cnt -- считаем TOTAL число всех вакансий
            FROM 
                public.employers as e,
                public.vacancies as v
            WHERE
                e.id = v.employer_id
        ) as t       
    GROUP BY employer, t.cnt
    ORDER BY cnt_pct DESC
    LIMIT 50 
    )
    
    UNION ALL
    
    (SELECT 
        'TOTAL' as employer, -- добавляем строку 
        SUM(cnt_pct)
    FROM 
        (SELECT 
            e.name as employer,
            COUNT(v.id)::numeric / t.cnt::numeric * 100 as cnt_pct -- считаем долю вакансий в регионе от общего числа в %%
        FROM 
            public.employers as e
        INNER JOIN
            public.vacancies as v
            ON e.id = v.employer_id  -- соединяем таблицы по id региона
        CROSS JOIN  -- прописываем TOTAL в каждую строку чтобы суметь посчитать %% в каждой строке
            (   
                SELECT 
                    COUNT(v.id) as cnt -- считаем TOTAL число всех вакансий
                FROM 
                    public.employers as e,
                    public.vacancies as v
                WHERE
                    e.id = v.employer_id
            ) as t       
        GROUP BY employer, t.cnt
        ORDER BY cnt_pct DESC
        LIMIT 50 
        ) as s
    )
'''

In [686]:
# результат запроса
df = pd.read_sql_query(query_4_2_plus, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\3184278282.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_2_plus, connection)


,employer,cnt_pct
0,Яндекс,3.929101
1,Ростелеком,0.998028
2,Тинькофф,0.902494
3,СБЕР,0.869972
4,Газпром нефть,0.672805
5,ИК СИБИНТЕК,0.664675
6,МТС,0.593532
7,DataArt,0.502063
8,Совкомбанк Технологии,0.414659
9,Первый Бит,0.357745


ПОсмотрим, сколько и каких есть вакансий, с ключевым навыком Python, и не попавших в отбор по набору масок DS. Видно, что фильтры явнов пропускают такие DS направления как Data Engineer, Data Analyst, Аналитик данных.

In [703]:
query_6_4_plus = f'''
    SELECT
        DISTINCT v.name,
        COUNT(v.id) as cnt
    FROM 
        public.vacancies as v
    WHERE
        LOWER(v.key_skills) LIKE '%python%'
        AND 
        NOT
        (
            LOWER(v.name) LIKE '%data scientist%' 
            OR LOWER(v.name) LIKE '%data science%'
            OR LOWER(v.name) LIKE '%исследователь данных%'
            OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
            OR LOWER(v.name) LIKE '%machine learning%'
            OR LOWER(v.name) LIKE '%машинн%обучен%'
        )
    GROUP BY v.name
    ORDER BY cnt DESC
    LIMIT 50
'''

In [704]:
# результат запроса
df = pd.read_sql_query(query_6_4_plus, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\828204651.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_4_plus, connection)


,name,cnt
0,Data Engineer,47
1,Python Developer,44
2,Data Analyst,43
3,Программист Python,34
4,Аналитик,32
5,Разработчик Python,32
6,Python разработчик,28
7,Аналитик данных,25
8,DevOps инженер,23
9,Python developer,23


Посмотрим у каких DS вакансий не указан python в качестве ключевого навыка. Видно, что выбранные маски ошибочно относят к DS специальистов в области QML и TinyML.

In [712]:
query_6_4_plus2 = f'''
    SELECT DISTINCT
        v.name,
        v.key_skills,
        COUNT(v.id) as cnt
    FROM 
        public.vacancies as v
    WHERE
        LOWER(v.key_skills) NOT LIKE '%python%'
        AND 
        (
            LOWER(v.name) LIKE '%data scientist%' 
            OR LOWER(v.name) LIKE '%data science%'
            OR LOWER(v.name) LIKE '%исследователь данных%'
            OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
            OR LOWER(v.name) LIKE '%machine learning%'
            OR LOWER(v.name) LIKE '%машинн%обучен%'
        )
    GROUP BY v.name, v.key_skills
    ORDER BY cnt DESC
    --LIMIT 50
'''

In [713]:
# результат запроса
df = pd.read_sql_query(query_6_4_plus2, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\3423916229.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_4_plus2, connection)


,name,key_skills,cnt
0,C++ разработчик (QML),Qt\tООП\tC++\tMS Visual Studio\tРазработка ком...,9
1,Junior Data Scientist (Валидация скоринговых м...,Оценка рисков\tMS PowerPoint\tСтатистический а...,9
2,Архитектор решений с опытом в Tiny ML,Git\tООП\tAtlassian Jira\tJava\tAtlassian Conf...,6
3,Data Scientist (Валидация скоринговых моделей),Оценка рисков\tMS PowerPoint\tСтатистический а...,4
4,Архитектор решений (с опытом в TinyML),TinyML\tEmbedded\tSolution Architect\tАрхитект...,4
5,C++ Machine Learning Engineer (Computer Vision),Computer Vision\tC/C++\tMachine learning,3
6,Data Scientist,SQL\tData Analysis\tdata science,2
7,Senior Data Scientist,Знание стека DS\tЗнание теории машинного обуче...,2
8,Разработчик С++/QML,Git\tQt\tLinux\tАнглийский язык\tXML,2
9,Analyst Data Scientist,Data Science\tAnalyst,1


Подсчитаем, сколько вакансий удовлетворяет выбранному набору масок DS

In [714]:
query_6_plus = f'''
    SELECT DISTINCT
        --v.name,
        --v.key_skills,
        COUNT(v.id) as cnt
    FROM 
        public.vacancies as v
    WHERE
        (
            LOWER(v.name) LIKE '%data scientist%' 
            OR LOWER(v.name) LIKE '%data science%'
            OR LOWER(v.name) LIKE '%исследователь данных%'
            OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
            OR LOWER(v.name) LIKE '%machine learning%'
            OR LOWER(v.name) LIKE '%машинн%обучен%'
        )
    --GROUP BY v.name, v.key_skills
    --ORDER BY cnt DESC
    --LIMIT 50
'''

In [715]:
# результат запроса
df = pd.read_sql_query(query_6_plus, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\2492160640.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_plus, connection)


,cnt
0,480


Подсчитаем, сколько вакансий удовлетворяет набору масок DS, если скорректировать его и убрать вакансии из областей QML и TinyML.

In [716]:
query_6_plus2 = f'''
    SELECT DISTINCT
        --v.name,
        --v.key_skills,
        COUNT(v.id) as cnt
    FROM 
        public.vacancies as v
    WHERE
        (
            LOWER(v.name) LIKE '%data scientist%' 
            OR LOWER(v.name) LIKE '%data science%'
            OR LOWER(v.name) LIKE '%исследователь данных%'
            OR (v.name LIKE '%ML%' 
                AND LOWER(v.name) NOT LIKE '%html%'
                AND LOWER(v.name) NOT LIKE '%qml%'
                AND LOWER(v.name) NOT LIKE '%tiny%ml%'
                )
            OR LOWER(v.name) LIKE '%machine learning%'
            OR LOWER(v.name) LIKE '%машинн%обучен%'
        )
    --GROUP BY v.name, v.key_skills
    --ORDER BY cnt DESC
    --LIMIT 50
'''

In [717]:
# результат запроса
df = pd.read_sql_query(query_6_plus2, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_212\1490997789.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_plus2, connection)


,cnt
0,456


### подведем итог исследования, обобщите выводы
### здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

Выводы по доп.исследованиям приведены в соответствующих разделах выше. 

Предварительный анализ показал достаточно высокое качество данных, репрезнтативность выборки, как в разрезе регионов, так и работодателей, наличие компаний с широким геограчиеским присутствием, преимущественную концентрацию вакансий в городах-миллиониках и региональных центрах. 

В контекксте DS выборка на два порядка меньше, чемобщее число вакансий, однако и она дает представление о специфике специалистов в этой области. Python, SQL, библиотеки/инструменты базирующиеся на python в списке навыков являются важным атрибутом. Видна явная и сильная зависимость заработной платы от опыта работы. 

В качестве плана для дальнейшего анализа можно:
 - оценить зависимость заработной платы от ключевых навыков, 
 - выделить какие еще навыки встречаются наиболее часто, 
 - оценить корреляцию между наличием тех или иных навыков и опытом работы
 - выделить возможные особые направления в общей массе DS вакансий, например управленческое направление, оценить насколько для них отличаются от средних по выборке набор навыков, зарплата, опыт.